
Перейити в гугл диск и загрузить в корень диска файл 

https://drive.google.com/drive/ 



In [ ]:
#@title ##**Имя видео файла** { display-mode: "form" }
file_name = input("Введите имя файла: ") or "input.mp4"

In [ ]:
#@title ##**Проверка GPU** { display-mode: "form" }
!nvidia-smi

In [ ]:
#@title ##**Клонирование репозитория** { display-mode: "form" }
%cd /content

!git clone https://github.com/megvii-research/NAFNet

In [ ]:
#@title ##**Установить библиотеку** { display-mode: "form" }
%cd /content/NAFNet

!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

In [ ]:
#@title ##**Импорт библиотеки** { display-mode: "form" }
%cd /content/NAFNet

import gdown

import torch

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os
from google.colab import files
import shutil
import glob

In [ ]:
#@title ##**Скачать модель** { display-mode: "form" }
%cd /content/NAFNet

# Deblur
if not os.path.exists("./experiments/pretrained_models/NAFNet-REDS-width64.pth"):
  gdown.download('https://drive.google.com/uc?id=14D4V4raNYIOhETfcuuLI3bGLB-OYIv6X', "./experiments/pretrained_models/", quiet=False)

In [ ]:
#@title ##**Подключение к гугл диску** { display-mode: "form" }
from google.colab import drive
drive.mount('/mnt/gdrive')


In [ ]:
#@title ##**Создать папки для изображений** { display-mode: "form" }
#если нужно продолжить предедущую обработку, то пропустите этот пункт

%cd /content/NAFNet

deblur_upload_folder = '/mnt/gdrive/MyDrive/deblur_input'
deblur_result_folder = '/mnt/gdrive/MyDrive/deblur_output'

if os.path.isdir(deblur_upload_folder):
    shutil.rmtree(deblur_upload_folder)
if os.path.isdir(deblur_result_folder):
    shutil.rmtree(deblur_result_folder)
os.makedirs(deblur_upload_folder)
os.makedirs(deblur_result_folder)


In [ ]:
#@title ##**Секвенция видео** { display-mode: "form" }
#если нужно продолжить предедущую обработку, то пропустите этот пункт
import os
import time
import cv2

path = '/mnt/gdrive/MyDrive/'
full_path = os.path.join(path, file_name)

frames_of_video = int(cv2.VideoCapture(full_path).get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture(full_path).get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture(full_path)
success, image = vidcap.read()
count = 0
success = True

# создаем буфер для хранения кадров
buffer = []
while success:
    buffer.append(image)
    success, image = vidcap.read()
    count += 1

# записываем изображения из буфера в папку
for i, image in enumerate(buffer):
    cv2.imwrite("/mnt/gdrive/MyDrive/deblur_input/frame%09d.jpg" % i, image)
    time.sleep(1/fps_of_video)


In [ ]:
#@title ##**Определение функции** { display-mode: "form" }
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('NAFNet output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      imwrite(sr_img, save_path)

def sr_display(LR_l, LR_r, SR_l, SR_r):
  h,w = SR_l.shape[:2]
  LR_l = cv2.resize(LR_l, (w,h), interpolation=cv2.INTER_CUBIC)
  LR_r = cv2.resize(LR_r, (w,h), interpolation=cv2.INTER_CUBIC)
  fig = plt.figure(figsize=(w//40, h//40))
  ax1 = fig.add_subplot(2, 2, 1) 
  plt.title('Input image (Left)', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(2, 2, 2)
  plt.title('NAFSSR output (Left)', fontsize=16)
  ax2.axis('off')
  ax1.imshow(LR_l)
  ax2.imshow(SR_l)

  ax3 = fig.add_subplot(2, 2, 3) 
  plt.title('Input image (Right)', fontsize=16)
  ax3.axis('off')
  ax4 = fig.add_subplot(2, 2, 4)
  plt.title('NAFSSR output (Right)', fontsize=16)
  ax4.axis('off')
  ax3.imshow(LR_r)
  ax4.imshow(SR_r)

  plt.subplots_adjust(wspace=0.04, hspace=0.04)

def stereo_image_inference(model, img_l, img_r, save_path):
      img = torch.cat([img_l, img_r], dim=0)
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      img_L = visuals['result'][:,:3]
      img_R = visuals['result'][:,3:]
      img_L, img_R = tensor2img([img_L, img_R])
      
      imwrite(img_L, save_path.format('L'))
      imwrite(img_R, save_path.format('R'))


In [ ]:
#@title ##**Построить модель** { display-mode: "form" }
%cd /content/NAFNet

opt_path = 'options/test/REDS/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

In [ ]:
#@title ##**Deblur изображений** { display-mode: "form" }
%cd /content/NAFNet
import glob
import os
import time
from tqdm import tqdm

deblur_upload_folder = '/mnt/gdrive/MyDrive/deblur_input'
deblur_result_folder = '/mnt/gdrive/MyDrive/deblur_output'



upload_files = set()
while not upload_files:
    try:
        upload_files = set(os.listdir(deblur_upload_folder))
    except OSError as e:
        print(f"Error: {e}, retrying...")
        time.sleep(2)

result_files = set(os.listdir(deblur_result_folder))

input_list = [file_path for file_path in glob.glob(os.path.join(deblur_upload_folder, '*'))
              if os.path.basename(file_path) not in result_files and os.path.basename(file_path) in upload_files]


for input_path in tqdm(input_list, desc="Processing images"):
    img_input = imread(input_path)
    inp = img2tensor(img_input)
    output_path = os.path.join(deblur_result_folder, os.path.basename(input_path))
    single_image_inference(NAFNet, inp, output_path)

output_list = sorted(glob.glob(os.path.join(deblur_result_folder, '*')))
for input_path, output_path in tqdm(zip(input_list, output_list), desc="Loading images"):
    img_input = imread(input_path)
    img_output = imread(output_path)

In [ ]:
#@title ##**Сборка видео** { display-mode: "form" }
import cv2
import os
from tqdm.notebook import tqdm

output_file = "/mnt/gdrive/MyDrive/output.mp4"
path = '/mnt/gdrive/MyDrive/'

full_path = os.path.join(path, file_name)

cap = cv2.VideoCapture(full_path)
fps_of_video = int(cap.get(cv2.CAP_PROP_FPS))
cap.release()

img_files = os.listdir(deblur_result_folder)
img_files.sort()

frame = cv2.imread(os.path.join(deblur_result_folder, img_files[0]))
height, width, _ = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_file, fourcc, fps_of_video, (width, height))

for img_file in tqdm(img_files, desc='Сборка видео'):
    img_path = os.path.join(deblur_result_folder, img_file)
    frame = cv2.imread(img_path)

    writer.write(frame)

writer.release()
print('Сборка завершена')

In [ ]:
#@title ##**Добавление звука** { display-mode: "form" }
video = VideoFileClip(output_file)

audio = AudioFileClip(full_path)

video = video.set_audio(audio)

video.write_videofile('/mnt/gdrive/MyDrive/output_with_audio.mp4')


In [ ]:
#@title ##**Скачать файл (опционально, быстерее можно скачать в гугл диске)** { display-mode: "form" }
from google.colab import files
files.download('/mnt/gdrive/MyDrive/output_with_audio.mp4')